Подключение библиотеки

In [1]:
import os
import logging

import psycopg2
import psycopg2.extensions
import pandas as pd

ModuleNotFoundError: No module named 'psycopg2'

Подключение к БД

In [2]:
params = {
    "host": os.environ['POSGRES_HOST'],
    "port": os.environ['POSGRES_PORT'],
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}

Запрос 1:
Составим ТОП-10 самых комерчески успешных фильмов.

In [8]:
sql_req1 = '''
select title,
       revenue-budget as NPV
from movies order by 2 desc limit 10
'''
cursor.execute(sql_req1)
conn.commit()
pd.DataFrame(
    [a for a in cursor.fetchall()],
    columns=['title', 'NPV']).to_csv(os.environ['PANDAS_EXPORT_FOLDER'] + 'req1.csv')


Запрос 2:
ТОП-10 самых лучших фильмов по мнению пользователей.

In [3]:
sql_req2 = '''
select movies.title,
       round(100*av_mrk)/100 as mark
from (select movieid,avg(rating) as av_mrk 
      from ratings group by movieid order by 1 desc) as sample
left join  movies on movieid=id order by 2 desc limit 10
'''
cursor.execute(sql_req2)
conn.commit()
pd.DataFrame(
    [a for a in cursor.fetchall()],
    columns=['title', 'mark']).to_csv(os.environ['PANDAS_EXPORT_FOLDER'] + 'req2.csv')

Запрос из представления 1:
Определим "попсовые" фильмы, неплохо заработавшие на старте, но позже получившие низкие оценки пользователей

In [4]:
sql_req4view1 = '''
select movies.title,
       round(100*av_mrk)/100 as mark,
       revenue-budget as NPV
from (select movieid,
             avg(rating) as av_mrk 
      from ratings group by movieid order by 1 desc) as sample
left join  movies on movieid=id where av_mrk<2 order by 3 desc limit 10
'''
cursor.execute(sql_req4view1)
conn.commit()
pd.DataFrame(
    [a for a in cursor.fetchall()],
    columns=['title','mark','NPV']).to_csv(os.environ['PANDAS_EXPORT_FOLDER'] + 'req4view1.csv')


Запрос из представления 2:
А теперь найдем 10 самых недооцененные при старте проката фильмов, т.е. тех которые при выпуске не набрали значительных кассовых сборов, однака, в последующем набрали популярность среди пользователей.

In [5]:
sql_req4view2 = '''
select movies.title,
       round(100*av_mrk)/100 as mark,
       revenue-budget as NPV
from (select movieid,
             avg(rating) as av_mrk
      from ratings group by movieid order by 1 desc) as sample
left join  movies on movieid=id where av_mrk>4 order by 3 limit 10
'''
cursor.execute(sql_req4view2)
conn.commit()
pd.DataFrame(
    [a for a in cursor.fetchall()],
    columns=['title','mark','NPV']).to_csv(os.environ['PANDAS_EXPORT_FOLDER'] + 'req4view2.csv')

Запрос 3:
Посмотрим какие тэги у фильмов попавших в предыдущую выборку (в запрос из представления 2) (первые 10).

In [7]:
sql_req3 = '''
select title,
       tags
from (select movies.title,
             round(100*av_mrk)/100 as mark,
             revenue-budget as NPV,
             movies.id
      from (select movieid,
                   avg(rating) as av_mrk
            from ratings group by movieid order by 1 desc) as sample
      left join  movies on movieid=id where av_mrk>4 order by 3 limit 10) as tab1
left join keywords on tab1.id=keywords.id
'''
cursor.execute(sql_req3)
conn.commit()
pd.DataFrame(
    [a for a in cursor.fetchall()],
    columns=['title', 'tags']).to_csv(os.environ['PANDAS_EXPORT_FOLDER'] + 'req3.csv')


Запрос 6:
Cоздание таблицы нормированных значений бюджета и логарифма бюджета.

In [6]:
sql_req6 = '''
select (CAST(budget AS float)-(select min(budget) from movies))/(select max(budget)-min(budget) from movies) as bg_norm,
       (log(CAST(budget AS float))-(select min(log(budget) from movies))/(select max(log(budget))-min(log(budget)) from movies) as lg_bg_norm 
from movies order by 1
'''
cursor.execute(sql_req6)
conn.commit()
pd.DataFrame(
    [a for a in cursor.fetchall()],
    columns=['bg_norm', 'lg_bg_norm']).to_csv(os.environ['PANDAS_EXPORT_FOLDER'] + 'req6.csv')
